# 📊 Sentiment and Emotion Analysis with FinBERT-Tone and RoBERTa Go-Emotions (Real Time News Implementation)
This notebook filters financial news headlines by selected markets, applies `yiyanghkust/finbert-tone` for sentiment analysis, and uses `roberta-base-go_emotions` for emotion detection.

In [1]:
# FINANCIAL NEWS SENTIMENT ANALYZER (FIXED VERSION)
# Enhanced to reliably detect financial markets

# [1] Install packages (if needed)
!pip install transformers torch pandas requests tqdm python-dotenv

# [2] Import libraries
from transformers import pipeline
import pandas as pd
import requests
from datetime import datetime, timedelta
from tqdm import tqdm
import os
from dotenv import load_dotenv

# Initialize environment
load_dotenv()

# [3] Financial Market Detection Config (UPDATED)
MARKET_KEYWORDS = {
    "Stocks": ["stock", "equity", "share", "S&P", "NASDAQ", "Dow", "index", "bull", "bear", "market"],
    "Forex": ["forex", "currency", "USD", "EUR", "GBP", "JPY", "FX", "exchange rate", "dollar", "yen"],
    "Commodities": ["oil", "gold", "silver", "commodity", "crude", "barrel", "ounce", "copper", "futures"],
    "Economy": ["GDP", "inflation", "CPI", "unemployment", "economic", "growth", "recession", "Fed", "ECB", "central bank"],
    "Crypto": ["bitcoin", "crypto", "blockchain", "BTC", "ETH", "digital currency", "token", "DeFi"],
    "Bonds": ["bond", "yield", "treasury", "debt", "10-year", "notes", "credit rating"]
}

# [4] Relaxed Financial Content Filter (FIXED)
def is_financial(text):
    """More inclusive financial content detection"""
    text_lower = text.lower()
    
    # Must contain at least 2 financial terms
    financial_terms = sum(
        1 for terms in MARKET_KEYWORDS.values() 
        for term in terms if term in text_lower
    )
    
    # Exclude non-financial content
    non_financial = any(
        term in text_lower 
        for term in ["sports", "entertainment", "celebrity", "movie", "music"]
    )
    
    return financial_terms >= 2 and not non_financial

# [5] News Fetcher with Better Filtering (FIXED)
def fetch_news():
    """Get news from multiple sources with improved filtering and date handling"""
    sources = [
        ("NewsAPI", "https://newsapi.org/v2/top-headlines?category=business&apiKey=" + os.getenv('NEWSAPI_KEY')),
        ("AlphaVantage", "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&apikey=" + os.getenv('ALPHAVANTAGE_KEY'))
    ]

    seen_texts = set()
    all_news = []

    for source_name, url in sources:
        try:
            response = requests.get(url, timeout=10)
            data = response.json()

            if source_name == "NewsAPI":
                for article in data.get('articles', []):
                    title = article.get('title', '')
                    description = article.get('description', '')
                    text = f"{title}. {description}".strip()
                    published = article.get('publishedAt', '')[:10]  # 'YYYY-MM-DD'

                    if is_financial(text) and text not in seen_texts:
                        seen_texts.add(text)
                        all_news.append({"text": text[:1000], "date": published})

            elif source_name == "AlphaVantage":
                for item in data.get('feed', []):
                    title = item.get('title', '')
                    summary = item.get('summary', '')
                    text = f"{title}. {summary}".strip()
                    published = item.get('time_published', '')[:8]  # 'YYYYMMDD'
                    published = f"{published[:4]}-{published[4:6]}-{published[6:]}"  # format to YYYY-MM-DD

                    if is_financial(text) and text not in seen_texts:
                        seen_texts.add(text)
                        all_news.append({"text": text[:1000], "date": published})

        except Exception as e:
            print(f"⚠️ {source_name} error: {str(e)}")

    return all_news

# [6] Initialize Models
print("Loading AI models...")
finbert = pipeline("text-classification", model="yiyanghkust/finbert-tone")
emotion = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions")

# [7] Enhanced Signal Generator
def get_signal(text):
    """Generate trading signal with market context"""
    try:
        sentiment = finbert(text[:512])[0]['label']
        emotion_label = emotion(text[:512])[0]['label']
        
        # Market-specific adjustments
        market = next(
            (mkt for mkt, terms in MARKET_KEYWORDS.items() 
             if any(term in text.lower() for term in terms)),
            "General"
        )
        
        # Score calculation
        score = {
            "Positive": 1.0,
            "Neutral": 0.0,
            "Negative": -1.0
        }[sentiment]
        
        # Emotion adjustments
        score += {
            "joy": 0.3, "optimism": 0.2, "excitement": 0.1,
            "fear": -0.3, "anger": -0.2, "annoyance": -0.1
        }.get(emotion_label.lower(), 0)
        
        # Market multipliers
        score *= {
            "Stocks": 1.2,
            "Crypto": 1.3,
            "Commodities": 1.1
        }.get(market, 1.0)
        
        # Generate signal
        if score >= 1.5: return "🟢 Strong Buy"
        elif score >= 0.7: return "🟢 Buy"
        elif score <= -1.5: return "🔴 Strong Sell"
        elif score <= -0.7: return "🟠 Sell"
        return "⚪ Neutral"
        
    except Exception as e:
        print(f"⚠️ Analysis error: {str(e)}")
        return "⚪ Neutral"

# [8] Main Analysis Function (FIXED)
def analyze_financial_markets():
    """Run complete market analysis"""
    print("\n🔍 Fetching financial news...")
    news_items = fetch_news()

    results = []
    for market, keywords in MARKET_KEYWORDS.items():
        market_news = [
            item for item in news_items
            if any(keyword in item["text"].lower() for keyword in keywords)
        ]

        print(f"\n📊 {market} Sector ({len(market_news)} articles)")
        for item in tqdm(market_news, desc=market):
            results.append({
                "Market": market,
                "Headline": item["text"][:100] + "..." if len(item["text"]) > 100 else item["text"],
                "Signal": get_signal(item["text"]),
                "Date": item["date"]
            })

    return pd.DataFrame(results)
# [9] Run Analysis
results_df = analyze_financial_markets()

# [10] Display Results
if not results_df.empty:
    print("\n📈 Financial Market Signals")
    print("="*50)
    display(results_df.style.apply(
        lambda x: ["background: darkgreen" if "Strong Buy" in v 
                  else "background: lightgreen" if "Buy" in v
                  else "background: darkred" if "Strong Sell" in v
                  else "background: salmon" if "Sell" in v 
                  else "" for v in x],
        subset=['Signal']
    ))
else:
    print("\n⚠️ No financial market data found - check your news sources")

# [11] Save Results
results_df.to_csv(f"market_signals_{datetime.now().strftime('%Y%m%d')}.csv", index=False)
print("\n✅ Analysis saved to CSV")


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\hazim\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading AI models...


Device set to use cpu
Device set to use cpu



🔍 Fetching financial news...

📊 Stocks Sector (20 articles)


Stocks: 100%|██████████| 20/20 [00:14<00:00,  1.40it/s]


...


📊 Bonds Sector (1 articles)


Bonds: 100%|██████████| 1/1 [00:00<00:00,  1.12it/s]



📈 Financial Market Signals


,Market,Headline,Signal,Date
0,Stocks,"Dow Jones Futures: Israel-Iran Attacks Hit Market; The Next AI Winners After Nvidia, Broadcom? - Inv...",🟢 Buy,2025-06-14
1,Stocks,Bitcoin price targets mushroom as traders bet on $140K+ this bull run. From golden crosses to ascend...,🟢 Buy,2025-06-15
2,Stocks,"Lucid Group Stock: Analysts Fear This 1 Problem Is ""More Consequential"" Than Investors Think. Lucid ...",🟢 Buy,2025-06-15
3,Stocks,These 2 Beaten-Down Dividend Stocks and This ETF Yield Over 4%. Here's Why They Are Worth Doubling U...,🟢 Buy,2025-06-15
4,Stocks,3 Reasons Why This Beaten-Down Growth Stock Could Trounce the S&P 500 in the Second Half of 2025. Ap...,🟢 Buy,2025-06-15
5,Stocks,1 Recent Flashing Signal With a Perfect Track Record Since 1972 Suggests the S&P 500 Index Will Soar...,🟢 Strong Buy,2025-06-15
6,Stocks,"Here's How Many Shares of Apple Stock You Should Own to Get $1,000 in Yearly Dividends. When people ...",🟢 Buy,2025-06-15
7,Stocks,Is the Artificial Intelligence Boost Fading for This Top Tech Stock?. When artificial intelligence (...,🟠 Sell,2025-06-15
8,Stocks,The Fundamental Problem With MicroStrategy's Bitcoin-Buying Plan. One of the hottest tech stocks in ...,🟠 Sell,2025-06-15
9,Stocks,2 Artificial Intelligence ( AI ) Cloud Stocks to Buy in June. Artificial intelligence ( AI ) is pr...,🟢 Buy,2025-06-15



✅ Analysis saved to CSV
